In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

from torchsample.modules import ModuleTrainer

from torchsample.callbacks import EarlyStopping
from torchsample.regularizers import L1Regularizer

from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch
import time

from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import warnings

pd.set_option('max_column', 0)
warnings.filterwarnings('ignore')

from full_data_load_ep import *
from data_processing import *
# from data_model import *

import joblib

In [30]:
# load database credentials and create connection
user, password, server, database, port = load_db_credentials()
engine = create_engine(f'postgresql://{user}:{password}@{server}:{port}/{database}')

In [57]:
print('--- Reading Data From Database ---')
# read data in from database
query = open('player_stats.sql', 'r').read()

skaters = pd.read_sql(query, engine)
team_stats = pd.read_sql(''' select * from team_stats  ''', engine)
info = pd.read_sql(''' select * from player_info ''', engine)

print('--- Engineering features ---')

# merge player information with player seasons
skaters = skaters.merge(info[info.date_of_birth.notnull()].drop('position', axis=1), on = ['playerid'])


--- Reading Data From Database ---
--- Engineering features ---


In [58]:
skaters[skaters.league == 'USHS-Prep'].year.unique()

array(['2011-2012', '2010-2011', '2009-2010', '2012-2013', '2013-2014',
       '2014-2015', '2015-2016', '2016-2017', '2017-2018', '2018-2019',
       '2019-2020'], dtype=object)

## v1

In [32]:
scoring = skaters.groupby(['league', 'year']).g.sum() / \
            skaters.groupby(['league', 'year']).gp.sum()

scoring = scoring.rename('deflator').to_frame()

scoring = scoring.merge(scoring[scoring.index.get_level_values('year') == '2019-2020'],
              on='league',
              suffixes=('', '_base'),
              how='left').set_index(scoring.index)

scoring['deflator'] /= scoring['deflator_base']

v1 = scoring.copy()

### V2

In [33]:
df = pd.read_sql('''select 
	league,
	year,
	sum(g) as g,
	max(gp) as gp,
	count(distinct teamid) teams,
	sum(g) / (max(gp) * count(distinct teamid)) as goals_per_game
from skater_stats ss 
where season_stage not like '%ET%'
and season_stage not like '%Cup%'
and season_stage not like '%JCWC%'
and season_stage not like '%PJHL%'
and season_stage not like '%Czech U16 2%' 
and season_stage not like '%Showcase%'
and season_stage not like '%ECC%'
and season_stage not like '% Q'
and season_stage not like 'MHL Supercup'
and league not in ('CCHL2')
group by 
1
,2
having max(gp) * count(distinct teamid) > 0
;
''', engine)


scoring = df.merge(df[df['year'] == '2019-2020'][['league', 'goals_per_game']],
              on='league',
              suffixes=('', '_base'),
              how='left')

scoring['deflator']  = scoring['goals_per_game'] / scoring['goals_per_game_base']

v2 = scoring.copy()

### compare

In [34]:
v2.head()

,league,year,g,gp,teams,goals_per_game,goals_per_game_base,deflator
0,AHL,2005-2006,6557.0,80.0,27,3.035648,2.84639,1.066491
1,AHL,2006-2007,6346.0,80.0,27,2.937963,2.84639,1.032172
2,AHL,2007-2008,6522.0,80.0,29,2.811207,2.84639,0.987639
3,AHL,2008-2009,6481.0,80.0,29,2.793534,2.84639,0.981431
4,AHL,2009-2010,6647.0,80.0,29,2.865086,2.84639,1.006568


In [35]:
v1.head()

deflator  deflator_base
league year                              
AHL    2009-2010  0.968058       0.166469
       2010-2011  0.951287       0.166469
       2011-2012  0.933298       0.166469
       2012-2013  0.920630       0.166469
       2013-2014  0.948261       0.166469

In [36]:
compare = v1.merge(v2, left_index=True, right_on = ['league', 'year'], suffixes=('_v1', '_v2'))

In [37]:
import plotly.express as px

fig = px.scatter(compare, x="deflator_v1", y="deflator_v2", trendline="ols",hover_data=['league', 'year'])
fig.show()

In [28]:
skaters[(skaters.league == 'CCHL')
       & (skaters.year == '2005-2006')]

,playerid,player,position,league,year,team,teamids,stages,gp,g,a,tp,gpg,apg,ppg,perc_team_g,perc_team_a,perc_team_tp,date_of_birth,height,place_of_birth,weight,nation,shoots,status,shortname,contract,rights,under_contract,draft_year,draft_round,draft_pick,draft_team,nhl_draft,catches,draft_year_eligible,load_date
16765,12073,Scott Campbell,"{W,C}",CCHL,2005-2006,[PEM],[2161],[CJHL],45.0,25.0,33.0,58.0,0.555556,0.733333,1.288889,0.103,0.083,0.091,1986-06-06,188.0,"Navan, ON, CAN",95.0,Canada,L,Retired,scott-campbell,None,None,None,None,None,None,None,None,None,2004.0,2020-11-01
19332,12382,Ryan Lasch,{RW},CCHL,2005-2006,[PEM],[2161],[CJHL],56.0,70.0,77.0,147.0,1.250000,1.375000,2.625000,0.233,0.155,0.184,1987-01-22,170.0,"Lake Forest, CA, USA",71.0,USA,R,None,ryan-lasch,Indef.,None,None,None,None,None,None,None,None,2005.0,2020-11-01
20165,12517,Wayne Simmonds,{RW},CCHL,2005-2006,[BRO],[3248],[CJHL],49.0,24.0,19.0,43.0,0.489796,0.387755,0.877551,0.176,0.093,0.126,1988-08-26,188.0,"Scarborough, ON, CAN",84.0,Canada,R,None,wayne-simmonds,20/21,Toronto Maple Leafs,True,2007,2,61,Los Angeles Kings,None,None,2006.0,2020-11-01
20595,12556,Simon Lacroix,{D},CCHL,2005-2006,[CUM],[3690],[CJHL],47.0,5.0,10.0,15.0,0.106383,0.212766,0.319149,0.039,0.053,0.047,1989-05-29,186.0,"Orléans, ON, CAN",78.0,Canada,R,Retired,simon-lacroix,None,None,None,2007,7,196,New York Islanders,None,None,2007.0,2020-11-01
20629,12559,Alain Goulet,{D},CCHL,2005-2006,[OTT],[3190],[CJHL],41.0,6.0,14.0,20.0,0.146341,0.341463,0.487805,0.041,0.058,0.051,1988-09-22,190.0,"Kapuskasing, ON, CAN",88.0,Canada,R,Retired,alain-goulet,None,None,None,2007,6,159,Boston Bruins,None,None,2007.0,2020-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168969,87506,Jérémi LaPierre,{C},CCHL,2005-2006,[CUM],[3690],[CJHL],56.0,17.0,39.0,56.0,0.303571,0.696429,1.000000,0.112,0.173,0.148,1986-04-13,182.0,"Hearst, ON, CAN",74.0,Canada,L,Retired,jeremi-lapierre,None,None,None,None,None,None,None,None,None,2004.0,2020-11-01
169622,88044,Brent Fallon,{D},CCHL,2005-2006,[COR],[3404],[CJHL],42.0,4.0,19.0,23.0,0.095238,0.452381,0.547619,0.028,0.097,0.068,1986-07-31,182.0,"Endicott, NY, USA",81.0,USA,L,Retired,brent-fallon,None,None,None,None,None,None,None,None,None,2004.0,2020-11-01
169623,88045,Graham Fallon,{RW},CCHL,2005-2006,[COR],[3404],[CJHL],36.0,22.0,14.0,36.0,0.611111,0.388889,1.000000,0.182,0.083,0.125,1986-07-31,183.0,"Endicott, NY, USA",90.0,USA,L,Retired,graham-fallon,None,None,None,None,None,None,None,None,None,2004.0,2020-11-01
169644,88065,Brian Marks,{D},CCHL,2005-2006,[BRO],[3248],[CJHL],55.0,1.0,7.0,8.0,0.018182,0.127273,0.145455,0.007,0.031,0.021,1987-04-13,180.0,"Brownville, NY, USA",84.0,USA,L,None,brian-marks,-,None,None,None,None,None,None,None,None,2005.0,2020-11-01


In [25]:
skaters[(skaters.league == 'USDP')
       & (skaters.year == '2008-2009')].g.sum()\
/\
skaters[(skaters.league == 'USDP')
       & (skaters.year == '2008-2009')].gp.sum()

0.21663201663201664

In [ ]:
#cchl 2005-2006, ushs-prep 2006-2009
